In [1]:
from keras import applications
from keras.models import *
from keras.applications import *
from keras.preprocessing import *

g_top_n=100   #前多少类预测值

dogs = [
 'n02085620','n02085782','n02085936','n02086079'
,'n02086240','n02086646','n02086910','n02087046'
,'n02087394','n02088094','n02088238','n02088364'
,'n02088466','n02088632','n02089078','n02089867'
,'n02089973','n02090379','n02090622','n02090721'
,'n02091032','n02091134','n02091244','n02091467'
,'n02091635','n02091831','n02092002','n02092339'
,'n02093256','n02093428','n02093647','n02093754'
,'n02093859','n02093991','n02094114','n02094258'
,'n02094433','n02095314','n02095570','n02095889'
,'n02096051','n02096177','n02096294','n02096437'
,'n02096585','n02097047','n02097130','n02097209'
,'n02097298','n02097474','n02097658','n02098105'
,'n02098286','n02098413','n02099267','n02099429'
,'n02099601','n02099712','n02099849','n02100236'
,'n02100583','n02100735','n02100877','n02101006'
,'n02101388','n02101556','n02102040','n02102177'
,'n02102318','n02102480','n02102973','n02104029'
,'n02104365','n02105056','n02105162','n02105251'
,'n02105412','n02105505','n02105641','n02105855'
,'n02106030','n02106166','n02106382','n02106550'
,'n02106662','n02107142','n02107312','n02107574'
,'n02107683','n02107908','n02108000','n02108089'
,'n02108422','n02108551','n02108915','n02109047'
,'n02109525','n02109961','n02110063','n02110185'
,'n02110341','n02110627','n02110806','n02110958'
,'n02111129','n02111277','n02111500','n02111889'
,'n02112018','n02112137','n02112350','n02112706'
,'n02113023','n02113186','n02113624','n02113712'
,'n02113799','n02113978']

cats=['n02123045','n02123159','n02123394','n02123597'
,'n02124075','n02125311','n02127052']


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


## 预测方法

In [2]:
#参考：https://zhuanlan.zhihu.com/p/26541005

#from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
#from keras.applications.inception_v3 import preprocess_input, decode_predictions

from keras.preprocessing.image import *
#from keras.applications.inception_v3 import preprocess_input, decode_predictions


def predict(model, image_path, target_size, top_n=10, pre_func=None):
    """Run model prediction on image
    Args:
      model: keras model
      #img: PIL format image
      image_path: file path of image
      target_size: (width, height) tuple
      top_n: # of top predictions to return
    Returns:
      list of predicted labels and their probabilities
    """
    #img = image.load_img(image_path, target_size=target_size)
    img = load_img(image_path, target_size=target_size)
    if img.size != target_size:
        img = img.resize(target_size)
    #x = image.img_to_array(img)
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    
    if pre_func:
        x = pre_func(x)

    preds = model.predict(x)
    #return decode_predictions(preds, top=top_n)[0]
    return decode_func(preds, top=top_n)[0]


In [3]:
import os
import shutil

def rmrf_mkdir(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)
    
# 是否同一类别: 在模型预测top_n中有同样的类型，则认为是同一类别
# class_type-'cat' or 'dog'
def is_same_class(model, target_size, image_path, class_type, pre_func=None):
    preds = predict(model, image_path, target_size=target_size, top_n=g_top_n, pre_func=pre_func)  
    if class_type == 'cat':
        BAGS=cats   
    elif class_type == 'dog':
        BAGS=dogs   
        
    for i in range(len(preds)):
        if preds[i][0] in BAGS:
            return True
    
    return False

def pick_not_same(model, target_size, image_path, out_path, class_type, pre_func=None):
    filenames = os.listdir(image_path+'/'+class_type)
    
    for filename in filenames:
        new_filename = image_path+'/'+class_type+'/'+filename
        if not is_same_class(model, target_size, new_filename, class_type, pre_func):
            out_path_file = out_path+'/'+filename
            print(out_path_file)
            shutil.copy(new_filename, out_path_file)  #复制判断为不同类的图片
    return
    

In [4]:
def pick_up_images(MODEL, image_size, pre_func=None):
    #base_model = MODEL(weights='imagenet', include_top=True)
    base_model = MODEL
    pick_not_same(base_model, image_size, predict_dir, anormal_cat_dir, 'cat', pre_func)
    pick_not_same(base_model, image_size, predict_dir, anormal_dog_dir, 'dog', pre_func)
    print("Mode:{} pick up over".format(MODEL.name))
    return
    

In [5]:
predict_dir = 'data/test_pick'       #对猫的图片进行预测，选出结果非猫的图片
predict_dog_dir = 'data/test_pick'   #对狗的图片进行预测，选出结果非狗的图片

anormal_cat_dir= 'data/anormal/cat'  #猫训练庥中,模型认为非猫的图片
anormal_dog_dir= 'data/anormal/dog'  #狗训练庥中,模型认为非狗的图片

rmrf_mkdir(anormal_cat_dir)
rmrf_mkdir(anormal_dog_dir)



In [6]:

#模型放在函数外加载,避免每次调参时,都要加载
InceptionV3_model = InceptionV3(weights='imagenet', include_top=True)
Xception_model = Xception(weights='imagenet', include_top=True)
ResNet50_model = ResNet50(weights='imagenet', include_top=True) 
#VGG16_model = VGG16(weights='imagenet', include_top=True)   #权重文件最大,500M


In [8]:
#pick_up_images(VGG16, (224, 224), vgg16.preprocess_input)
#pick_up_images(InceptionV3, (299, 299), inception_v3.preprocess_input)
#pick_up_images(Xception, (299, 299), xception.preprocess_input)
#pick_up_images(ResNet50, (224, 224))

#decode_func=inception_v3.decode_predictions
#pick_up_images(InceptionV3_model, (299, 299), inception_v3.preprocess_input)

decode_func=xception.decode_predictions
pick_up_images(Xception_model, (299, 299), xception.preprocess_input)

#decode_func=resnet50.decode_predictions
#pick_up_images(ResNet50_model, (224, 224), resnet50.preprocess_input)

#decode_func=vgg16.decode_predictions
#pick_up_images(VGG16_model, (224, 224), vgg16.preprocess_input)


Mode:inception_v3 pick up over
data/anormal/cat/cat.9.jpg
data/anormal/cat/cat.54.jpg
data/anormal/cat/cat.63.jpg
data/anormal/cat/cat.14.jpg
data/anormal/cat/cat.46.jpg
data/anormal/cat/cat.8.jpg
data/anormal/cat/cat.15.jpg
data/anormal/cat/cat.49.jpg
data/anormal/cat/cat.27.jpg
data/anormal/cat/cat.37.jpg
data/anormal/cat/cat.61.jpg
data/anormal/cat/cat.38.jpg


KeyboardInterrupt: 